In [1]:
#pragma cling add_include_path("./thirdparty/kdb/include")
#pragma cling add_include_path("./thirdparty/kdb/include/internal")
#pragma cling load("./thirdparty/kdb/build/libkdb.so")
#pragma cling add_include_path("./thirdparty/libtorch/include")
#pragma cling add_include_path("./thirdparty/libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("./thirdparty/libtorch/lib")
#pragma cling load("libtorch")

#pragma cling add_include_path("./thirdparty/nest/include")

#include <torch/torch.h>
#include <torch/csrc/autograd/variable.h>
#include <torch/csrc/autograd/function.h>

#include "xtensor/xbuilder.hpp"
#include "xtensor/xarray.hpp"
#include "xtensor/xtensor.hpp"
#include "nest.h"
#include <vector>
#include <array>

#include </home/thorad/Core/Projects/Qenv/notebooks/thirdparty/kdb/include/kdb_cpp.h>
#include <string>
#include <iostream>
#include <typeinfo>

kdb::Connector kcon;
#define HOST_ADDR "127.0.0.1"
#define HOST_PORT 5050

if (!kcon.connect(HOST_ADDR, HOST_PORT))
        return -1;

kdb::Result rs0 = kcon.sync(".server.DumStep[((0;1);(1;5);(2;4);(3;5));0]");
kdb::Result rs1 = kcon.sync(".server.DumStep[((0;1);(1;5);(2;4);(3;5));1]");

K rks0 = rs0.get_res();
K rks1 = rs1.get_res();

// std::cout << rs1 << std::endl;

typedef nest::Nest<torch::Tensor> TensorNest;

In [2]:
std::vector<kdb::Result> responses = {};

In [3]:
%timeit -n 1000 responses.push_back(kcon.sync(".server.DumStep[((0;1);(1;5);(2;4);(3;5));0]"));

339 us +- 29.8 us per loop (mean +- std. dev. of 7 runs 1000 loops each)


In [2]:
rs0

@0x7f11b13bc078

In [ ]:
K elementOne = kK(rks1)[0]; 

In [3]:
std::cout << rks1->n << std::endl;

4


In [2]:
std::vector({1,2,3})

{ 1, 2, 3 }

In [4]:

TensorNest k_list_to_nest(K k_list) {
        
};


// `agentId`observation`reward`done`episode_step`episode_return
std::vector<TensorNest> derive_multistep_from_result(kdb::Result& result){
    K kres  = result.get_res();
    std::vector<TensorNest> tensors;
    for (int i=0; i< kres->n; i++) {
        std::cout << i << std::endl;
        K step = kK(kres)[0]; // TODO
        tensors.push_back(TensorNest(std::vector({
//             std::move(TensorNest(torch::full({1,1},kF(step[1])))), // observation // TODO
            std::move(TensorNest(torch::full({1,1},kK(step)[2]->f,torch::dtype(torch::kFloat64)))), // reward
            std::move(TensorNest(torch::full({1,1},kK(step)[3]->g,torch::dtype(torch::kBool)))), // done
            std::move(TensorNest(torch::full({1,1},kK(step)[4]->f,torch::dtype(torch::kFloat64)))), // episode_step TODO change
            std::move(TensorNest(torch::full({1,1},kK(step)[5]->f,torch::dtype(torch::kFloat64))))  // episode_return
            })));
    };
    return tensors;
};




In [5]:
std::vector<TensorNest> res = derive_multistep_from_result(rs1);

0
1
2
3


In [5]:
std::cout << res.size() << std::endl;

2


In [8]:
for (auto const it : column) {
    std::cout << it << ' ';
}

139954981177984 139954981178016 139954981178048 139954981178112 

input_line_7:2:22: error: use of undeclared identifier 'column'
 std::cout << typeid(column).name() << std::endl;
                     ^
input_line_7:2:7: error: no member named 'cout' in namespace 'std'; did you mean 'count'?
 std::cout << typeid(column).name() << std::endl;
 ~~~~~^~~~
      count
/opt/anaconda3/envs/cling/bin/../lib/gcc/x86_64-conda-linux-gnu/7.5.0/../../../../x86_64-conda-linux-gnu/include/c++/7.5.0/bits/stl_algo.h:4076:5: note: 'count' declared here
    count(_InputIterator __first, _InputIterator __last, const _Tp& __value)
    ^


Interpreter Error: 

In [9]:
kdb::Result res = kcon.sync("select from tbl_test");

In [10]:
kdb::Table tbl = res.get_table();

In [11]:
// Traverse a column
kdb::Vector<kdb::Type::Long> column = tbl.get_column<kdb::Type::Long>(0);
std::cout << "=======================================\nColumn 0: ";
for (auto const &it : column) {
    std::cout << it << ' ';
}
std::cout << '\n';

Column 0: 1 2 3 4 


In [ ]:
std::cout << torch::full({1,1},1) << std::endl;

IncrementalExecutor::executeFunction: symbol '_ZN3kdb9ConnectorD1Ev' unresolved while linking function '_GLOBAL__sub_I_cling_module_10'!
You are probably missing the definition of kdb::Connector::~Connector()
Maybe you need to load the corresponding shared library?


In [2]:
typedef nest::Nest<torch::Tensor> TensorNest;

In [3]:
torch::Tensor x = torch::ones({2, 2}, torch::requires_grad());
torch::Tensor y = torch::randn({2, 2});

In [2]:
std::vector<size_t> shape = { 3, 2, 4 };
std::vector<size_t> strides = { 8, 4, 1 };

In [4]:
std::vector<size_t> shape = { 3, 2, 4 };
xt::xarray<double, xt::layout_type::row_major> a(shape);

In [7]:
std::array<size_t, 3> shape = { 3, 2, 4 };
xt::xtensor<double, 3> b(shape);

In [ ]:
// typedef int  I;          // Already defined within k.h
// typedef char *S;         // Already defined within k.h
// I khpu(S hostname, I portNumber, S usernamePassword)
khpu ("localhost", 12345, "username:password");